### 1. Data Source

The dataset used in this article comes from Alibaba Cloud Tianchi:
Alibaba Cloud Tianchi - Audience Expansion Dataset
This dataset contains three tables, which respectively record the activity of two marketing strategies on Alipay:

`emb_tb_2.csv`: User feature dataset

`effect_tb.csv`: Advertising click dataset

`seed_cand_tb.csv`: User type dataset

This analysis report mainly uses the advertising click data, with the following fields:

 - dmp_id: Marketing strategy code (the original data document does not provide an explanation, so based on the data, it is set as 1: Control group, 2: Marketing Strategy One, 3: Marketing Strategy Two)
 - user_id: Alipay user ID
 - label: Whether the user clicked on the activity ad on that day (0: Not clicked, 1: Clicked)

### 2. Data Processing
#### 2.1 Data Import and Cleaning

##### 2.1.1 Table Integration

In [2]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [7]:
# load data
data = pd.read_csv('../data/effect_tb.csv',header = None)
data.columns = ["dt","user_id","label","dmp_id"]

# removing dt column since it's useless
data = data.drop(columns = "dt")
data.head()

,user_id,label,dmp_id
0,1,0,1
1,1000004,0,1
2,1000004,0,2
3,1000006,0,1
4,1000006,0,3


In [8]:
# table summary
data.describe()

,user_id,label,dmp_id
count,2.645958e+06,2.645958e+06,2.645958e+06
mean,3.112995e+06,1.456297e-02,1.395761e+00
std,1.828262e+06,1.197952e-01,6.920480e-01
min,1.000000e+00,0.000000e+00,1.000000e+00
25%,1.526772e+06,0.000000e+00,1.000000e+00
50%,3.062184e+06,0.000000e+00,1.000000e+00
75%,4.721132e+06,0.000000e+00,2.000000e+00
max,6.265402e+06,1.000000e+00,3.000000e+00


##### 2.1.2 Handling duplicate

In [10]:
# shape of df
data.shape

(2645958, 3)

In [9]:
# distinct count of columns
data.nunique()

user_id    2410683
label            2
dmp_id           3
dtype: int64

In [12]:
# The number of the rows are the the number of the independent users 
# are inconsistent, check it there are any duplicates existing

# Return a subset of data that contains all rows that are duplicated 
# (including both the first occurrence and subsequent duplicates).
data[data.duplicated(keep = False)].sort_values(by = ["user_id"])

,user_id,label,dmp_id
8529,1027,0,1
1485546,1027,0,1
1579415,1471,0,1
127827,1471,0,1
404862,2468,0,1
...,...,...,...
1382121,6264633,0,1
1382245,6264940,0,1
2575140,6264940,0,1
1382306,6265082,0,3


In [13]:
# drop duplicate
data = data.drop_duplicates()

# check if any duplicates left
data[data.duplicated(keep = False)]

,user_id,label,dmp_id


##### 2.1.3 Handling Missing Values

In [15]:
# check null values
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2632975 entries, 0 to 2645957
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   user_id  int64
 1   label    int64
 2   dmp_id   int64
dtypes: int64(3)
memory usage: 80.4 MB


No null values were found, no need to handle missing values.


##### 2.1.4 checking outliers

In [16]:
data.head()

,user_id,label,dmp_id
0,1,0,1
1,1000004,0,1
2,1000004,0,2
3,1000006,0,1
4,1000006,0,3


In [17]:
data.pivot_table(index = "dmp_id", columns = "label", values = "user_id",
                aggfunc = "count", margins = True)

label,0,1,All
dmp_id,,,
1,1881745,23918,1905663
2,404811,6296,411107
3,307923,8282,316205
All,2594479,38496,2632975


No outliers were found in variables "dmp_id" and "Label". No need to handle outliers.

##### 2.1.5 Check data type

In [18]:
data.dtypes

user_id    int64
label      int64
dmp_id     int64
dtype: object

#### 2.2 Sample Size Calculation


Sample Size Calculation is a statistical process used to determine the number of observations or participants required in a study to ensure that the results are statistically significant. The goal is to ensure that, given a desired confidence level and statistical power, the study has enough data to detect the effect you're investigating.

Key Elements of Sample Size Calculation:

`Effect Size`: The magnitude of the difference or effect you expect to detect in the study. The larger the effect size, the smaller the sample size required.

`Significance Level (α)`: Typically set at 0.05, this is the probability of rejecting the null hypothesis when it is actually true (Type I error).

`Statistical Power (1-β)`: The probability of correctly rejecting the null hypothesis (i.e., detecting an effect if there is one). It is usually set at 80% or 90%. The higher the power, the larger the sample size needed.

`Population Standard Deviation`: The variability in the population or data you're studying. Higher variability means you'll need a larger sample size to detect an effect.

`Type of Hypothesis Test`: The sample size calculation depends on the type of statistical analysis being performed, such as a t-test, chi-square test, or regression analysis.

Steps to Perform a Sample Size Calculation:

 - Define the study objective and the expected effect size.

 - Set the significance level (α) and statistical power (1-β).

 - Based on the selected statistical test and expected effect size, calculate the required sample size.

Use the calculator on thin website: https://www.evanmiller.org/ab-testing/sample-size.html

In [ ]:
# a. Calculate the click rate of control group 1.26% --> set it as the base CRT
data[data["dmp_id"] == 1]["label"].mean()

# b. set the minimum detectable effec size as 1% (The Minimum Detectable 
# Effect is the smallest effect that will be detected (1-β)% of the time.)
 

# c. Set Statistical power 1−β = 80% (Percent of the time the minimum 
# effect size will be detected, assuming it exists)

# d. Set Significance level α = 0.05 (Percent of the time a difference 
# will be detected, assuming one does NOT exist)

## Obtained Sample Size ---> 2167

In [19]:
# sample size of campaigns
data["dmp_id"].value_counts()

dmp_id
1    1905663
2     411107
3     316205
Name: count, dtype: int64

The sample sizes of both of the treatment strategy groups are over 2167, meeting the minimum sample size requirement.

In [20]:
# Save the clean data for future use
# save it to file
data.to_csv("../data/output.csv", index = False)

# reload data
# data = pd.read_csv("data/output.csv")